In [ ]:
HOME = "../.." # path to the root of the project two levels up

# Activate the environment
using Pkg
Pkg.activate(HOME)
Pkg.precompile()

using IceFloeTracker: get_tiles, get_tile_dims, load
using ColorTypes: RGBA

In [ ]:
img = load("bering_strait/truecolor/20200504.aqua.truecolor.250m.tiff")

In [ ]:
# Say we want to split the image into roughly 8x8 tiles
size(img) .÷ 8

### Tile size
Let us choose tiles of size 450x450. To do that use the `get_tiles` function.

In [ ]:
# Let's view the documentation for the function
@info @doc get_tiles

In [ ]:
tiles = get_tiles(img, 450)

In [ ]:
# Top left tile dimensions
get_tile_dims(tiles[1])

In [ ]:
# Bottom right tile dimensions
get_tile_dims(tiles[end])

Note that the bottom right tile has been extended to cover the rest of the image.

In [ ]:
# View the first tile
img[tiles[1]...]

Further, note that the tiles on the right and bottom edges are slightly bigger.

### View the full tiling

In [ ]:
function build_tiling_illustration(img, side_length)

    # Create new canvas to draw on
    newimg = similar(img, RGBA{Float64})

    # Apply transparency to the tiles
    for tile_coords in get_tiles(img, side_length)
        tile = @view img[tile_coords...]
        alpha = rand(0.5:0.05:1)
        transparent_tile = map(c -> RGBA(c.r, c.g, c.b, alpha), tile)
        newimg[tile_coords...] .= transparent_tile
    end

    # View the image
    newimg
end

build_tiling_illustration(img, 450)


### Optimal tile side length

Perhaps there is a better way to split the image into tiles. We can use the `get_optimal_tile_size` function to determine a better tiling given an initial tile side length.

In [ ]:
using IceFloeTracker: get_optimal_tile_size

@info @doc get_optimal_tile_size

In [ ]:
best_side_length = get_optimal_tile_size(450, size(img))

As expected, a tile size of 450 is not optimal for this image. The function get_optimal_tile_size suggests a fitter tiling is possible using tiles of 451 pixels in side length for this image.

In [ ]:
build_tiling_illustration(img, 451)

In [ ]:
# Area of corner tile with side length 451
get_tile_dims(get_tiles(img, 451)[end]) |> prod

Note that the area of the corner tile for the 450-tiling is larger.

In [ ]:
get_tile_dims(get_tiles(img, 450)[end]) |> prod